<a href="https://colab.research.google.com/github/1900690/yolo-bbox-area-caliculate/blob/main/area_caliculate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install ultralytics  # install
from ultralytics import YOLO, checks, hub

checks()  # checks

Ultralytics 8.3.37 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.5/107.7 GB disk)


In [2]:
from ultralytics.utils.plotting import Annotator
from ultralytics import YOLO
import cv2
import os
import tensorflow as tf
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd

model = YOLO('/content/itigo.pt')  # ★モデルを指定

if os.path.exists('/content/detect.txt'):
  os.remove('/content/detect.txt')

# Process the image
source = cv2.imread('/content/itigo2.jpg')#★検出画像を指定
results = model(source,conf=0.25,iou=0.45,save_txt=True)

# Extract results
annotator = Annotator(source, example=model.names)

for box in results[0].boxes.xyxy.cpu():
    width, height, area = annotator.get_bbox_dimension(box)
    datalist = [str(box[0].item())," ",str(box[1].item())," ",str(box[2].item())," ",str(box[3].item())," ",str(area.item()),"\n"]
    f = open('/content/myfile.txt', 'a')
    f.writelines(datalist)
    f.close()

#計算する座標データを読み込み
txt_file = pd.read_csv("/content/myfile.txt",header=None, sep=" ")
#大きいほうから〇個分を抽出
pickup_file=txt_file.nlargest(5, columns=4)#★検出個数を指定
# テキスト形式で出力
pickup_file.to_csv('/content/pickup.txt',header=False, index=False, sep=" ")

#書き込む画像を読み込み
im_original = Image.open("/content/itigo2.jpg")
image_height,image_width=np.array(im_original).shape[:2]

#書き込む座標データを読み込み
txt_file = pd.read_csv("/content/pickup.txt",header=None, sep=" ")
for annotation in range(len(txt_file)):
  draw = ImageDraw.Draw(im_original)
  x1=txt_file[0][annotation]
  y1=txt_file[1][annotation]
  x2=txt_file[2][annotation]
  y2=txt_file[3][annotation]
  draw.rectangle((x1, y1, x2, y2), outline=(0, 255, 0), width=5)

  im_original.save('/content/itigo1_after.jpg')


0: 640x640 49 strawberrys, 4076.6ms
Speed: 20.3ms preprocess, 4076.6ms inference, 37.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict
1 label saved to runs/detect/predict/labels


FileNotFoundError: [Errno 2] No such file or directory: '/content/itigo1.jpg'